In [111]:
import torch
from tqdm import tqdm
import numpy as np

from itertools import chain

In [3]:
FILENAME = "first_names.all.txt"
N = 3

In [4]:
names = []
with open(FILENAME, "r") as f:
    for line in f:
        line = line.strip().lower()
        if not line or not line.isalpha():
            continue
        names.append(line)

chars = set()
for name in names:
    chars.update(name.strip())

len(chars), "".join(sorted(chars))

(87,
 'abcdefghijklmnopqrstuvwxyzßàáâãäåæçèéêëìíîïðñòóôõöøùúûüýþāăąćčďđēėęěğģīıĳķļľłņňőřşţťūżž')

In [5]:
before_start_chars = list(reversed([f"-{i}" for i in range(1, N + 1)]))
end_char = "<END>"

char_to_code = {
    c: i for i, c in
    enumerate([end_char] + before_start_chars + sorted(chars))
}
code_to_char = {
    v: k for k, v in char_to_code.items()
}

chars_total = len(char_to_code)

chars_total, list(char_to_code.items())[:N + 5]

(91,
 [('<END>', 0),
  ('-3', 1),
  ('-2', 2),
  ('-1', 3),
  ('a', 4),
  ('b', 5),
  ('c', 6),
  ('d', 7)])

In [9]:
dataset = []
for name in tqdm(names):
    lst = before_start_chars + list(name) + [end_char]
    codes = [char_to_code[c] for c in lst]
    dataset.append(np.array(codes))

100%|██████████| 157710/157710 [00:00<00:00, 342123.96it/s]


In [140]:
shape = [chars_total] * (N + 1)
stat = np.full(shape, 0.01)
stat.shape

(91, 91, 91, 91)

In [141]:
for name in tqdm(dataset):
    for pos in range(len(name) - N):
        index = tuple(name[pos : pos + N + 1])
        stat[index] += 1


100%|██████████| 157710/157710 [00:03<00:00, 50566.50it/s]


In [144]:
def generate_name() -> str:
    idx = [char_to_code[c] for c in before_start_chars]
    while True:
        next_char_distribution = stat[tuple(idx)]
        next_char = np.random.choice(
            len(next_char_distribution),
            p=next_char_distribution / next_char_distribution.sum(),
        )

        if next_char in [char_to_code[c] for c in chain(before_start_chars, [end_char])]:
            break

        yield next_char
        idx = idx[1:] + [next_char]



In [148]:
for _ in range(9):
    print("".join(code_to_char[i] for i in list(generate_name())))

nell
swalde
yanashit
shealachesietavernhowley
goda
deja
dygňżðøîbúèyeőâūvđòóúdčīžäęőágùxfťěņečhùýiőïhāñu
breya
rexhmir
